In [1]:
from pymantic import sparql # type:ignore
from rdflib import Graph # type:ignore

# Blazegraph 서버와 연결
server = sparql.SPARQLServer('http://172.30.1.98:9999/blazegraph/namespace/AIModel/sparql')

In [2]:
# Create a graph object
g = Graph()

# Parse the RDF data from a .ttl file
file_path = 'model_data.ttl' # Replace with the path to your .ttl file
g.parse(file_path, format="turtle")

# Dictionary to map predicates to human-readable labels
predicate_map = {}
# Iterate over all triples and check for relevant predicates

for subject, predicate, obj in g:
    # Check if the predicate is in our mapping
    predicate_str = str(predicate)
    pre1 = predicate_str.replace('http://knowledgemap.kr/koai/def/type/','')
    pre2 = predicate_str.replace('http://knowledgemap.kr/koai/def/type/','koai:')
    
    # Add the mapping to the output dictionary
    predicate_map[pre1] = pre2 

# Remove the specified key from the dictionary
key_to_remove = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'

# Check if the key exists and delete it
if key_to_remove in predicate_map:
    del predicate_map[key_to_remove]

In [3]:
def create_sparql_query(model_name, predicate, select_var):
    """
    동적으로 SPARQL 쿼리를 생성하는 함수
    :param model_name: 모델 이름 (예: HHJingbo/Bo)
    :param predicate: SPARQL에서 사용할 Predicate (예: koai:isFineTunedFrom)
    :param select_var: SELECT 절에 사용할 변수 (예: ?model, ?downloads 등)
    :return: 생성된 SPARQL 쿼리 문자열
    """
    return f'''
    PREFIX koai: <http://knowledgemap.kr/koai/def/type/>

    SELECT {select_var}
    WHERE {{
        koai:{model_name} {predicate} {select_var} .
    }}
    '''

def execute_query(query):
    """
    Executes the SPARQL query and returns the result.
    :param query: The SPARQL query to execute.
    :return: The result of the query.
    """
    result = server.query(query)  # SPARQL endpoint 요청
    return result['results']['bindings']


def get_available_metadata(model_name):
    """
    모델에 어떤 메타데이터가 있는지 확인하는 함수
    :param model_name: 모델 이름
    :return: 모델에서 존재하는 메타데이터 항목 목록
    """
    # 모델에 대해 확인할 수 있는 메타데이터
    metadata = predicate_map
    
    available_metadata = []

    # 각 메타데이터 항목에 대해 존재하는지 확인하고 목록에 추가
    for category, predicate in metadata.items():
        sparql_query = create_sparql_query(model_name, predicate, "?value")
        results = execute_query(sparql_query)
        
        if results:
            available_metadata.append(category)

    return available_metadata


def query_model_info(model_name):
    """
    모델 정보에 대한 다양한 메타데이터를 조회하여 출력하는 함수
    :param model_name: 모델 이름 (예: HHJingbo_Bo)
    """
    # 모델에서 사용 가능한 메타데이터 항목을 가져오기
    available_metadata = get_available_metadata(model_name)
    
    # 사용자가 조회할 메타데이터 항목을 선택하도록 안내
    print(f"모델 '{model_name}'에서 조회할 수 있는 메타데이터 항목:")
    for i, category in enumerate(available_metadata, 1):
        print(f"{i}. {category}")

    selected_index = int(input(f"조회할 메타데이터 항목을 선택하세요 (1-{len(available_metadata)}): "))
    
    if selected_index < 1 or selected_index > len(available_metadata):
        print("잘못된 입력입니다. 다시 시도해주세요.")
        return
    
    selected_category = available_metadata[selected_index - 1]

    # 선택된 항목에 대해 SPARQL 쿼리 실행
    metadata_dict = predicate_map

    predicate = metadata_dict[selected_category]
    sparql_query = create_sparql_query(model_name, predicate, "?value")
    results = execute_query(sparql_query)

    print(f"\n{selected_category}:")
    if results:
        for result in results:
            for key, value in result.items():
                print(f"{key.capitalize()}: {value['value']}")
    else:
        print("No data found.")
    

# 모델명을 입력받아 메타데이터를 검색하는 함수 실행
model_name = input("모델명을 입력하세요 (예: HHJingbo_Bo): ")
query_model_info(model_name)

모델 'albert_albert_base_v1'에서 조회할 수 있는 메타데이터 항목:
1. tags
2. private
3. downloads
4. created_at
5. hasFineTunedModel
6. Purpose
7. arxiv
8. pipeline_tag
9. region
10. usesDataset
11. library_name
12. ModelArchitecture
13. license
14. id
15. gated
16. last_modified
17. likes
18. language
19. author

private:
Value: false
